In [1]:
import os
import requests
from pprint import pprint
from tqdm import tqdm

In [12]:
host = "http://0.0.0.0:8890/correct"

def call_gec(text):
    data = {
        "text": text,
        'iterations': 3,
        'min_probability': 0.7,
        'min_error_probability': 0.5,
        'add_spell_check': False,
    }
    resp = requests.post(host, json=data)
    res = resp.json()
    return res

text = "WHo is yourr daddy?"
res = call_gec(text)
print(res['debug_info'])

<Iteration 1>
Sentence Error Probability: 0.8968254327774048

#### Before ####
WHo is yourr daddy ?

#### After #####
WHo is yourr daddy ?

[Model Correction]
--------------------------------------------------------------------------------
__START__                     $KEEP                         0.9907964468002319
WHo                           $REPLACE_Who                  0.551882266998291
is                            $KEEP                         0.9691675901412964
yourr                         $REPLACE_you                  0.31872689723968506
daddy                         $KEEP                         0.8281116485595703
?                             $KEEP                         0.9950874447822571
--------------------------------------------------------------------------------





In [40]:
ref_m2_files = [
#     '/home/citao/Edison_workspace/Edison_tmp_data/autocorrect/bea2019/wi+locness/m2/A.dev.gold.bea19.m2',
#     '/home/citao/Edison_workspace/Edison_tmp_data/autocorrect/bea2019/wi+locness/m2/B.dev.gold.bea19.m2',
#     '/home/citao/Edison_workspace/Edison_tmp_data/autocorrect/bea2019/wi+locness/m2/C.dev.gold.bea19.m2',
#     '/home/citao/Edison_workspace/Edison_tmp_data/autocorrect/bea2019/wi+locness/m2/N.dev.gold.bea19.m2',
#     '/home/citao/Edison_workspace/Edison_tmp_data/autocorrect/bea2019/wi+locness/m2/ABC.train.gold.bea19.m2',
#     '/home/citao/Edison_workspace/Edison_tmp_data/autocorrect/bea2019/wi+locness/m2/ABCN.dev.gold.bea19.m2',
    '/home/citao/Edison_workspace/Edison_tmp_data/autocorrect/bea2019/wi+locness/m2/ABCN.all.gold.bea19.m2'
    
]

ref_m2 = []
for rmf in ref_m2_files:
    ref_m2.extend(open(rmf).read().split('\n\n'))

orig_sents = [i.split('\n')[0].lstrip('S ') for i in ref_m2]

In [18]:
orig_file = './output/original.txt'

cor_file = './output/wil_corrected_iter4_spell_check.txt'
cor_m2_file = './output/wil_corrected_iter4_spell_check.m2'

# cor_file = './output/wil_corrected_spell_check.txt'
# cor_m2_file = './output/wil_corrected_spell_check.m2'

of = open(orig_file, 'w')
cf = open(cor_file, 'w')

for sent in tqdm(orig_sents):
    res = call_gec(sent, True)
    of.write(sent+'\n')
    cf.write(res['data']+'\n')
    
of.close()
cf.close()

100%|██████████| 38693/38693 [1:01:21<00:00, 10.51it/s]


In [19]:
py_env_bin = '/home/citao/anaconda2/envs/python37/bin/'
errant_parallel_bin = os.path.join(py_env_bin, 'errant_parallel')
shell_cmd = "{} -orig {} -cor {} -out {}".format(errant_parallel_bin, orig_file, cor_file, cor_m2_file)
print(shell_cmd)
print(os.popen(shell_cmd).read())

/home/citao/anaconda2/envs/python37/bin/errant_parallel -orig ./output/original.txt -cor ./output/wil_corrected_iter4_spell_check.txt -out ./output/wil_corrected_iter4_spell_check.m2
Loading resources...
Processing parallel files...



In [27]:
cor_file = './output/wil_corrected_iter5_spell_check.txt'
cor_m2_file = './output/wil_corrected_iter5_spell_check.m2'
ref_m2_file = '/home/citao/Edison_workspace/Edison_tmp_data/autocorrect/bea2019/wi+locness/m2/ABCN.all.gold.bea19.m2'

errant_compare_bin = os.path.join(py_env_bin, 'errant_compare')
shell_cmd = "{} -hyp {} -ref {} -cat 1".format(errant_compare_bin, cor_m2_file, ref_m2_file)
print(os.popen(shell_cmd).read())

shell_cmd = "{} -hyp {} -ref {} -cat 2".format(errant_compare_bin, cor_m2_file, ref_m2_file)
print(os.popen(shell_cmd).read())

# shell_cmd = "{} -hyp {} -ref {} -cat 3 -dt".format(errant_compare_bin, cor_m2_file, ref_m2_file)
# print(os.popen(shell_cmd).read())



===================== Span-Based Correction ======================
Category       TP       FP       FN       P        R        F0.5
M              9018     5500     9095     0.6212   0.4979   0.5918
R              18763    17999    25033    0.5104   0.4284   0.4916
U              3725     3233     3861     0.5354   0.491    0.5259

=========== Span-Based Correction ============
TP	FP	FN	Prec	Rec	F0.5
31506	26732	37989	0.541	0.4534	0.5209



===================== Span-Based Correction ======================
Category       TP       FP       FN       P        R        F0.5
ADJ            244      625      838      0.2808   0.2255   0.2677
ADJ:FORM       88       36       81       0.7097   0.5207   0.6617
ADV            262      524      814      0.3333   0.2435   0.3104
CONJ           49       52       319      0.4851   0.1332   0.3174
CONTR          108      84       112      0.5625   0.4909   0.5466
DET            4755     2210     3206     0.6827   0.5973   0.6637
MORPH          611  

In [38]:
wrong_cases = [
    "If I had it to choose between both transportation I think I probably chose car because it is better for me to go by car then to go by bus .",
    "In my opinion I think that public transport in future is going to continue to be used because there are a lot of people that does n't have enough money to pay for a car even if it is used , so public transport continue to have future but people are going to use more the cars then bus or underground .",
    "But i was so mad at him and so ansiouns to make his life imposible , and soon my fear of death and my anger for all of the sufering i had been throug became stronger and bigger , i had made a desition i was goingo to do it , if he dedicated 4 years of his life tourtoring me and not wanting me to be happy i would the time is nesesary for him to have a miserable life and i wo nt stop until i had acomplished my goald",
    "econd reason . in the English syntaxis , the structure for construction paragraph or sentences , the verb is written before the subject , but no always , Which is the rule ?",
]

for c in wrong_cases:
    res = call_gec(c)
    print(c, '\n')
    print(res['data'], '\n')
    pprint(res['corrections'])
    
    print()


If I had it to choose between both transportation I think I probably chose car because it is better for me to go by car then to go by bus . 

If I had to choose between both transportation I think I probably choose a car because it is better for me to go by car than to go by bus . 

[['it to', 'to', [9, 14]],
 ['chose', 'choose a', [69, 74]],
 ['then', 'than', [120, 124]]]

In my opinion I think that public transport in future is going to continue to be used because there are a lot of people that does n't have enough money to pay for a car even if it is used , so public transport continue to have future but people are going to use more the cars then bus or underground . 

I think that public transport in the future is going to continue to be used because there are a lot of people that do n't have enough money to pay for a car even if it is used , so public transport will continue to have a future , but people are going to use more the cars then bus or underground . 

[['In my opinion I

In [65]:
call_gec("My namme is Citao .", True)

{'status': True,
 'data': 'My name is Ci tao .',
 'corrections': [['namme', 'name', [3, 8]], ['Citao', 'Ci tao', [12, 17]]],
 'orig_tokens': [['My', 'namme', 'is', 'Citao', '.']],
 'cor_tokens': [['My', 'name', 'is', 'Ci tao', '.']]}

In [66]:
call_gec("My namme is Citao .", False)

{'status': True,
 'data': 'My name is Citao .',
 'corrections': [['namme', 'name', [3, 8]]],
 'orig_tokens': [['My', 'namme', 'is', 'Citao', '.']],
 'cor_tokens': [['My', 'name', 'is', 'Citao', '.']]}